<a href="https://colab.research.google.com/github/vatturisrinivas/op_project/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 🚀 Simple GAN in PyTorch (Google Colab Ready)
# Generates data following: y = 2x, z = 4x pattern
# ============================================

# Install PyTorch if needed (uncomment next line in case your runtime lacks it)
# !pip install torch torchvision torchaudio --quiet

import torch
import torch.nn as nn
from tqdm import tqdm  # For progress bar

# --------------------------------------------
# ✅ Step 1: Device setup (GPU or CPU)
# --------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --------------------------------------------
# ✅ Step 2: Generate synthetic training data
# --------------------------------------------
n = 1000
first_column = torch.rand(n, 1).to(device)
data = torch.cat([first_column, 2 * first_column, 4 * first_column], dim=1)

# --------------------------------------------
# ✅ Step 3: Define the Generator
# --------------------------------------------
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 50),
            nn.ReLU(),
            nn.Linear(50, 3)
        )

    def forward(self, x):
        return self.model(x)

# --------------------------------------------
# ✅ Step 4: Define the Discriminator
# --------------------------------------------
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# --------------------------------------------
# ✅ Step 5: Initialize models, loss, and optimizers
# --------------------------------------------
generator = Generator().to(device)
discriminator = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.001)

# --------------------------------------------
# ✅ Step 6: Train the GAN
# --------------------------------------------
epochs = 5000

for epoch in tqdm(range(epochs)):
    # --- Train Discriminator ---
    optimizer_d.zero_grad()
    real_labels = torch.ones(n, 1).to(device)
    fake_labels = torch.zeros(n, 1).to(device)

    # Real data loss
    d_loss_real = criterion(discriminator(data), real_labels)

    # Fake data loss
    noise = torch.randn(n, 3).to(device)
    fake_data = generator(noise)
    d_loss_fake = criterion(discriminator(fake_data.detach()), fake_labels)

    # Total discriminator loss
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    optimizer_d.step()

    # --- Train Generator ---
    optimizer_g.zero_grad()
    g_loss = criterion(discriminator(fake_data), real_labels)
    g_loss.backward()
    optimizer_g.step()

    # Print progress every 1000 epochs
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

# --------------------------------------------
# ✅ Step 7: Generate and view synthetic results
# --------------------------------------------
with torch.no_grad():
    generated_data = generator(torch.randn(n, 3).to(device)).cpu().numpy()

print("\nGenerated Data (First 10 rows):")
for i in range(10):
    print(generated_data[i])

# --------------------------------------------
# ✅ Step 8: Validate results
# --------------------------------------------
print("\nValidation (For the first 10 rows):")
for i in range(10):
    first = generated_data[i][0]
    second = generated_data[i][1]
    third = generated_data[i][2]
    print(f"First: {first:.4f}, Expected Second: {2 * first:.4f}, Actual Second: {second:.4f}")
    print(f"Second: {second:.4f}, Expected Third: {2 * second:.4f}, Actual Third: {third:.4f}\n")


Using device: cpu


 21%|██        | 1052/5000 [00:03<00:14, 279.79it/s]

Epoch [1000/5000] | D Loss: 1.3884 | G Loss: 0.6938


 41%|████      | 2043/5000 [00:08<00:10, 278.24it/s]

Epoch [2000/5000] | D Loss: 1.3805 | G Loss: 0.6916


...

Epoch [3000/5000] | D Loss: 1.3848 | G Loss: 0.6962


 81%|████████  | 4040/5000 [00:15<00:03, 291.85it/s]

Epoch [4000/5000] | D Loss: 1.3577 | G Loss: 0.6566


100%|██████████| 5000/5000 [00:18<00:00, 263.99it/s]

Epoch [5000/5000] | D Loss: 1.3858 | G Loss: 0.6967

Generated Data (First 10 rows):
[0.41372493 0.7921966  1.5774162 ]
[0.2449942  0.47502047 0.9511551 ]
[0.27691168 0.5338634  1.0507252 ]
[0.4539718 0.8903095 1.7596691]
[0.56608963 1.1110386  2.2029305 ]
[0.24880259 0.48494005 0.9545137 ]
[0.5109309 0.9978832 1.9743557]
[0.68051136 1.325095   2.6078417 ]
[0.18354301 0.36118054 0.707551  ]
[0.6297597 1.1329279 2.193567 ]

Validation (For the first 10 rows):
First: 0.4137, Expected Second: 0.8274, Actual Second: 0.7922
Second: 0.7922, Expected Third: 1.5844, Actual Third: 1.5774

First: 0.2450, Expected Second: 0.4900, Actual Second: 0.4750
Second: 0.4750, Expected Third: 0.9500, Actual Third: 0.9512

First: 0.2769, Expected Second: 0.5538, Actual Second: 0.5339
Second: 0.5339, Expected Third: 1.0677, Actual Third: 1.0507

First: 0.4540, Expected Second: 0.9079, Actual Second: 0.8903
Second: 0.8903, Expected Third: 1.7806, Actual Third: 1.7597

First: 0.5661, Expected Second: 1.1322, A